In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np
import os
import sys

from tqdm import tqdm

from resnet import ResNet18
from data_aug import collect_advs, AdvDataSet

GeForce RTX 2070 SUPER


In [2]:
if torch.cuda.is_available() == True:
    device = torch.device('cuda:0')
    print(torch.cuda.get_device_name())
else:
    device = torch.device('cpu')
device

GeForce RTX 2070 SUPER


device(type='cuda', index=0)

In [3]:
batch_size = 128
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

def load_data():    
    transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip(), 
                                    transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
    transform_test = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_test)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)
    
    dataset = {'train': trainset, 'test': testset}
    data_loader = {'train': train_loader, 'test': test_loader}
    return dataset, data_loader

def load_iter(data_loader, data_type):
    bar_format = '{bar:30} {n_fmt}/{total_fmt} [{elapsed}<{remaining} {rate_fmt}] {desc}'
    
    if data_type == 'train':
        train_loader = data_loader['train']
        train_iter = tqdm(enumerate(train_loader), total=len(train_loader), unit_scale=batch_size, bar_format=bar_format)
        return train_iter
    elif data_type == 'test':
        test_loader = data_loader['test']
        test_iter = tqdm(enumerate(test_loader), total=len(test_loader), unit_scale=batch_size, bar_format=bar_format)
        return test_iter
    else:
        print('Data Error!!!')

In [4]:
def imshow(img):
    img[0] = img[0] * 0.2023 + 0.4914
    img[1] = img[1] * 0.1994 + 0.4822
    img[2] = img[2] * 0.2010 + 0.4465
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [5]:
def train(model):
    model.train()
    train_loss = 0
    total = 0
    correct = 0
    step = 0
    train_iter = load_iter(data_loader, 'train')
    for i, (batch, label) in train_iter:
        batch, label = batch.to(device), label.to(device)
        output = model(batch)
        
        optimizer.zero_grad()
        loss = loss_function(output, label)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = output.max(1)
        total += label.size(0)
        correct += predicted.eq(label).sum().item()

        acc = 100. * correct / total
        train_iter.set_description(f'[{acc:.2f}% ({correct}/{total})]', True)

In [6]:
def test(model):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    acc = 0.
    test_iter = load_iter(data_loader, 'test')

    for i, (batch, label) in test_iter:
        batch, label = batch.to(device), label.to(device)
        output = model(batch)
        loss = loss_function(output, label)

        test_loss += loss.item()
        _, predicted = output.max(1)
        total += label.size(0)
        correct += predicted.eq(label).sum().item()

        acc = 100. * correct / total
        test_iter.set_description(f'[{acc:.2f}%({correct}/{total})]', True)
    return acc

In [7]:
def save_model(epoch, acc, optimizer):
    global best_acc
    if acc > best_acc:
        state = {
            'model': model.state_dict(),
            'acc': acc,
            'epoch': epoch,
            'optimizer': optimizer.state_dict()
        }
        if not os.path.isdir('models'):
            os.mkdir('models')
        torch.save(state, f'./models/model_{epoch}.pth')
        best_acc = acc
        print('Saving Model...')

def load_model(name):
    state_dict = torch.load(f'./models/{name}.pth', map_location=device)
    model = ResNet18()
    model.to(device)
    model.load_state_dict(state_dict['model'])
    optimizer = optim.SGD(model.parameters(), lr=1e-1, momentum=0.9, weight_decay=1e-4)
#     optimizer.load_state_dict(state_dict['optimizer'])
    return model, optimizer

In [17]:
# Training
model = ResNet18()
model.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-1, momentum=0.9, weight_decay=1e-4)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[25, 35])
best_acc = 0
dataset, data_loader = load_data()
for epoch in range(1, 51):
    print(f'Epoch {epoch}')
    train(model)
    test_acc = test(model)
    save_model(epoch, test_acc, optimizer)
    scheduler.step()

Files already downloaded and verified
Files already downloaded and verified


                               128/50048 [00:00<00:49 999.77it/s] [8.59% (11/128)]: 

Epoch 1


██████████████████████████████ 50048/50048 [00:46<00:00 1066.55it/s] [21.78% (10891/50000)]: 
██████████████████████████████ 10112/10112 [00:03<00:00 3303.98it/s] [28.91%(2891/10000)]: 
                               128/50048 [00:00<00:49 1015.64it/s] [34.38% (44/128)]: 

Saving Model...
Epoch 2


...

Saving Model...
Epoch 32


██████████████████████████████ 50048/50048 [00:44<00:00 1120.25it/s] [100.00% (49999/50000)]: 
██████████████████████████████ 10112/10112 [00:02<00:00 3485.57it/s] [84.49%(8449/10000)]: 
                               128/50048 [00:00<00:46 1079.88it/s] [100.00% (128/128)]: 

Epoch 33


███▍                           5632/50048 [00:05<00:41 1065.34it/s] [100.00% (5632/5632)]: 


KeyboardInterrupt: 

In [ ]:
# Validation
model, optimizer = load_model('baseline')
loss_function = nn.CrossEntropyLoss()
test(model)

## Adversarial Training

In [9]:
model, optimizer = load_model('baseline_2')
epsilon = 0.01
dataset, data_loader = load_data()
adv_instances = collect_advs(model, data_loader, epsilon)

Files already downloaded and verified
Files already downloaded and verified


██████████████████████████████ 50048/50048 [02:51<00:00 291.48it/s] [# of Collected Adv Instances : 10239]: 


In [11]:
advset = AdvDataSet(adv_instances)
advset.slicing(42, 2000)
trainset = dataset['train']
advset_list = advset.concat_dataset(trainset)
len(advset)

10239

In [12]:
from torch.utils.data import Dataset, ConcatDataset
a = ConcatDataset([advset, trainset])
len(a)

60239

In [22]:
# Training
model = ResNet18()
model.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-1, momentum=0.9, weight_decay=1e-4)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[25, 35])
best_acc = 0
data_loader['train'] = torch.utils.data.DataLoader(advset_list[4], batch_size=batch_size, shuffle=True)
for epoch in range(1, 51):
    print(f'Epoch {epoch}')
    train(model)
    test_acc = test(model)
    save_model(epoch, test_acc, optimizer)
    scheduler.step()

                               128/52096 [00:00<00:52 992.03it/s] [15.62% (20/128)]: 

Epoch 1


██████████████████████████████ 52096/52096 [00:48<00:00 1074.23it/s] [46.91% (24391/52000)]: 
██████████████████████████████ 10112/10112 [00:02<00:00 3575.04it/s] [52.28%(5228/10000)]: 
                               128/52096 [00:00<00:47 1103.20it/s] [61.72% (79/128)]: 

Saving Model...
Epoch 2


██████████████████████████████ 52096/52096 [00:47<00:00 1100.88it/s] [68.27% (35501/52000)]: 
██████████████████████████████ 10112/10112 [00:02<00:00 3572.76it/s] [70.07%(7007/10000)]: 
                               128/52096 [00:00<00:47 1084.51it/s] [72.66% (93/128)]: 

...

██████████████████████████████ 52096/52096 [00:44<00:00 1171.26it/s] [100.00% (52000/52000)]: 
██████████████████████████████ 10112/10112 [00:02<00:00 3686.09it/s] [86.84%(8684/10000)]: 
                               128/52096 [00:00<00:45 1142.61it/s] [100.00% (128/128)]: 

Epoch 49


██████████████████████████████ 52096/52096 [00:44<00:00 1182.09it/s] [100.00% (52000/52000)]: 
██████████████████████████████ 10112/10112 [00:02<00:00 3687.00it/s] [86.89%(8689/10000)]: 
                               128/52096 [00:00<00:44 1174.05it/s] [100.00% (128/128)]: 

Epoch 50


██████████████████████████████ 52096/52096 [00:44<00:00 1181.55it/s] [100.00% (52000/52000)]: 
██████████████████████████████ 10112/10112 [00:02<00:00 3673.58it/s] [86.89%(8689/10000)]: 


In [24]:
model_list = ['baseline_2']
model_list += [f'slice_00{i}' for i in range(5)]
for m in model_list:
    state_dict = torch.load(f'models/{m}.pth')
    print(m, state_dict['acc'])

baseline_2 85.53
slice_000 86.45
slice_001 86.89
slice_002 87.24
slice_003 86.99
slice_004 87.12
